In [ ]:
# https://jakevdp.github.io/blog/2013/06/01/ipython-notebook-javascript-python-communication/
# https://github.com/fluxtream/fluxtream-ipy/
#   blob/master/Communication%20between%20kernel%20and%20javascript%20in%20iPython%202.0.ipynb

In [1]:
from IPython.display import HTML
from game import crush
from game import crush_ui
reload(crush)
reload(crush_ui)

javascript = """
<div id="board_10_14"></div>
<script type="text/Javascript">
create_board("#board_10_14",10,14,5);
</script>
"""
#HTML(crush_ui.javascript_test)
HTML(crush_ui.javascript_base+javascript)

In [2]:
from game import crush, crush_ui
reload(crush)
reload(crush_ui)
board = crush.new_board(10, 14, n_colours=5)
HTML(crush_ui.display_via_javascript_script("#board_10_14", board))